好的，"佈局"是台灣用詞的意思是"佈局"。




In [8]:
import csv
import os
from datetime import datetime

def parse_item(line):
    line = line.strip()
    if line.startswith("Item('"):
        term_start = line.find("Item('") + 6
        term_end = line.find("'", term_start)
        term = line[term_start:term_end]

        translations_start = line.find("'", term_end + 1) + 1
        translations_end = line.rfind("')")
        translations = line[translations_start:translations_end].split(';')
        
        translation_dict = {}
        for t in translations:
            t = t.strip()
            if '=>' in t:
                key, rest = t.split('=>')
                if ':' in rest:
                    lang, trans = rest.split(':')
                    lang = lang.strip()
                    trans = trans.strip()
                else:
                    continue
            elif ':' in t:
                key, trans = t.split(':')
                lang = key.strip()
                trans = trans.strip()
            else:
                continue
            
            if lang == 'zh-hans':
                lang = 'zh-cn'
            elif lang == 'zh-hant':
                lang = 'zh-tw'
                
            translation_dict[lang] = trans
        
        return term, translation_dict
    return None, None

def process_file(file_path, output_path):
    term_dict = {}
    all_languages = set(['en'])
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            term, translations = parse_item(line)
            if term:
                if term not in term_dict:
                    term_dict[term] = {'en': term}
                for key in translations:
                    term_dict[term][key] = translations[key]
                all_languages.update(translations.keys())

    language_order = ['en', 'zh-tw', 'zh-cn', 'zh-hk'] + sorted(all_languages - set(['en', 'zh-tw', 'zh-cn', 'zh-hk']))

    output_filename = os.path.basename(file_path).replace('.txt', '_Data.csv')
    output_full_path = os.path.join(output_path, output_filename)
    
    with open(output_full_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=language_order)
        writer.writeheader()
        for term_info in term_dict.values():
            writer.writerow(term_info)

    print(f"CSV file {output_filename} has been created successfully in {output_path}.")

def process_all_files(input_dir, output_dir):
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.txt'):
            file_path = os.path.join(input_dir, file_name)
            process_file(file_path, output_dir)

# 目錄設定
input_dir = 'wikiRawData'
output_dir = 'outputData'

# 確保輸出目錄存在
os.makedirs(output_dir, exist_ok=True)

# 處理所有文件
process_all_files(input_dir, output_dir)


CSV file CGroup_IT_Data.csv has been created successfully in outputData.
CSV file CGroup_Electronics_Data.csv has been created successfully in outputData.
CSV file CGroup_IT_Temp_Data.csv has been created successfully in outputData.
